# This is an office location recommender!

#### Let's import our createMap.py that contains generateMap function:

In [110]:
import src.createMap as cm

#### Create a Pandas dataframe with our final dataset. It contains the ideal locations for our new office and also the locations of all services we want to have close to us.

In [111]:
all_clean = pd.read_csv('./output/all_clean.csv')

#### Count the ideal how many ideal locations we have per city:

In [112]:
city_count = all_clean.groupby(['city']).count()
city_sorted = city_count.sort_values(['tech_startups'], ascending=[False])

In [113]:
city_sorted['tech_startups']

city
New York               52
San Francisco          52
Seattle                10
Boston                  6
Chicago                 5
Paris                   5
Toronto                 5
Mountain View           4
Palo Alto               4
Los Angeles             4
London                  3
Boulder                 3
Bellevue                3
Montreal                3
Reston                  2
Redwood City            2
Sydney                  2
Menlo Park              2
Amsterdam               2
Copenhagen              2
Berlin                  2
Corte Madera            2
Colorado Springs        1
Birmingham              1
Sunnyvale               1
Stockholm               1
Singapore               1
Budapest                1
Santa Monica            1
Santa Clara             1
San Mateo               1
San Jose                1
Cambridge               1
Lancaster               1
Honolulu                1
Pittsburgh              1
Emeryville              1
Olpe                    1
Glen Co

#### Let's choose a city and save it on a variable:

In [114]:
choose_city = str(input("Insert a city from the list above: ")).capitalize()

Insert a city from the list above: Budapest


In [116]:
if choose_city in city_sorted['tech_startups']:
    office_map = cm.generateMap(choose_city)
else:
    print("The city have chosen is not listed above. Please choose again.")

### Finally, the locations get painted on a map :)

In [117]:
office_map